In [1]:
import json, csv
import math
import os
import numpy as np
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool
from gensim import corpora, models, similarities
import jieba

In [2]:
inverted_filename = 'inverted_file.json'
url2content_name = 'url2content.json'
url2title_name = 'url2titles.json'
standlexi_name = 'extern/lexicon_mixed.json'
stoplist_name = 'extern/stop_list.json'

doc2url_name = 'news_data_1/NC_1.csv'
training_name = 'news_data_1/TD.csv'
query_name = 'news_data_1/QS_1.csv'

outcsv_name = 'out.csv'

In [3]:
num_threads = 16
MAXCAND = 300
DOCSIZE = 100000
num_topics = 100

In [4]:
test = False

In [5]:
MAXGRAM = 2

In [6]:
print("loading")
inverted = json.load(open(inverted_filename, 'r'))
stop_list = json.load(open(stoplist_name, 'r'))
urlcontents = json.load(open(url2content_name, 'r'))
urltitles = json.load(open(url2title_name, 'r'))
print("done")

loading
done


In [7]:
#load text into corpera
raw_documents = []
doc_index = {}

with open(doc2url_name, 'r') as f:
    for i, line in tqdm(enumerate(f), total=DOCSIZE):
        if i == 0:
            print(line)
        else:
            docname, url = line.strip().split(',') # doc_id, url
            d_id = len(raw_documents)
            doc_index[docname] = d_id
            text = urlcontents[url]+urltitles[url]
            raw_documents.append(text)

index2doc = {a:b for b, a in doc_index.items()}

News_Index,News_URL




In [8]:
# warmup
list(jieba.cut("Hello world !"))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/home/student/05/b05902064/.local/lib/python3.7/site-packages/jieba/__init__.py", line 152, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmp0w0eo5dg' -> '/tmp/jieba.cache'
Loading model cost 1.512 seconds.
Prefix dict has been built succesfully.


['Hello', ' ', 'world', ' ', '!']

In [9]:
# use jieba to cut the sentences
def taskseg(s):
    it = jieba.cut(s)
    return [w for w in it if w in inverted and w not in stop_list]

with Pool(num_threads) as p:
    chunksize = 100
    raw_documents = list(tqdm(p.imap(taskseg, raw_documents, chunksize=chunksize), total=DOCSIZE))

In [ ]:
# or use inverted file to build the documents
# raw_documents = []
# doc_index = {}
# doc2url = {}
# with open(doc2url_name, 'r') as f:
#     for i, line in tqdm(enumerate(f), total=DOCSIZE):
#         if i == 0:
#             print(line)
#         else:
#             docname, url = line.strip().split(',') # doc_id, url
#             d_id = len(raw_documents)
#             doc_index[docname] = d_id
#             text = urlcontents[url]+urltitles[url]
#             raw_documents.append(text)

In [10]:
# build a dictionary
dictionary = corpora.Dictionary(raw_documents)

In [11]:
#build the corpus
corpus = [dictionary.doc2bow(text) for text in raw_documents]

In [12]:
# tfidf_model = models.TfidfModel(corpus)
# corpus_tfidf = tfidf_model[corpus]

In [13]:
# lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=num_topics)
# corpus_lsi = lsi[corpus_tfidf] 

In [ ]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=num_topics)
corpus_lsi = lsi[corpus_tfidf] 

In [14]:
lsi.print_topics(10)

[(0,
  '0.111*"台灣" + 0.098*"％" + 0.088*"（" + 0.088*"）" + 0.088*"." + 0.087*"年" + 0.085*"政府" + 0.083*"陳" + 0.080*"：" + 0.080*"馬"'),
 (1,
  '0.204*"." + 0.171*"％" + -0.149*"公投" + -0.149*"課綱" + 0.145*"元" + -0.137*"黨" + 0.134*"市場" + 0.130*"億元" + 0.127*"台股" + 0.122*"美元"'),
 (2,
  '0.483*"巨蛋" + 0.435*"遠雄" + 0.333*"市府" + 0.182*"柯文" + -0.152*"公投" + 0.140*"大" + 0.125*"停工" + 0.123*"北" + -0.109*"黨" + 0.101*"哲"'),
 (3,
  '0.439*"課綱" + 0.338*"教育部" + 0.245*"學生" + -0.220*"公投" + 0.151*"高中" + 0.146*"微調" + 0.131*"教育" + -0.119*"巨蛋" + -0.118*"黨" + 0.115*"學校"'),
 (4,
  '-0.314*"課綱" + -0.214*"教育部" + -0.172*"巨蛋" + -0.163*"遠雄" + 0.153*"水扁" + 0.150*"扁" + -0.149*"％" + 0.145*"保外" + 0.142*"陳" + -0.127*"公投"'),
 (5,
  '-0.373*"Uber" + 0.217*"水扁" + 0.212*"扁" + 0.210*"中油" + 0.207*"保外" + 0.204*"元" + 0.186*"油價" + 0.172*"陳" + 0.151*"公升" + 0.134*"就醫"'),
 (6,
  '0.244*"中油" + 0.210*"公投" + -0.203*"水扁" + -0.199*"課綱" + -0.193*"扁" + -0.191*"保外" + 0.185*"Uber" + 0.183*"油價" + 0.180*"元" + 0.158*"公升"'),
 (7,
  '-0.581*"Uber" + 0.2

In [15]:
index = similarities.MatrixSimilarity(corpus_lsi)
index.num_best = MAXCAND

In [16]:
def match(q):
    q = taskseg(q)
    vec_bow = dictionary.doc2bow(q)
    vec_lsi = lsi[vec_bow]    
    sims = index[vec_lsi]
#     print(sims)
    return [(index2doc[i],s) for i,s in sims]

In [17]:
if not test:
    num_train = 20
    train_scores = {}
    train_qlist = []
    with open(training_name, 'r') as f:
        for i, line in enumerate(f):
            if i == 0:
                print(line)
            else:
                fields = line.strip().split(',') # q_id, text
                q, d, score = fields

                score = int(score)

                if q not in train_scores:
                    train_scores[q] = {d:score}                
                    train_qlist.append(q)
                else:
                    train_scores[q][d] = score 
    # print("done")
    train_qlist = train_qlist[:num_train]
    print(len(train_qlist))

Query,News_Index,Relevance

20


In [18]:
if not test:
    mean = 0

    for qtext in train_qlist:        
        scores = match(qtext)

        myrank = [train_scores[qtext][d] if d in train_scores[qtext] else 0 for d, s in scores]
        perfrank = sorted(myrank)[::-1]

        my_dcg = 0
        for i,r in enumerate(myrank):
            if i == 0:
                my_dcg += r
            else:
                my_dcg += r/math.log(i+1, 2)

        perf_dcg = 0
        for i,r in enumerate(perfrank):
            if i == 0:
                perf_dcg += r
            else:
                perf_dcg += r/math.log(i+1, 2)

        cur = my_dcg/(perf_dcg+1e-8)
        print(cur)
        mean += cur

    print("[NDCG]", mean / len(train_qlist))

0.4332192861775118
0.6416222623149652
0.5233603044544168
0.42783989871103506
0.5123652579068676
0.4612807963030497
0.5973245331901924
0.6983666654794483
0.6440772777154382
0.8418262178323148
0.0
0.5684266210012836
0.5395295470163508
0.8048895774057374
0.6902311423456654
0.5949593105455783
0.5442795859280566
0.8338370815572085
0.7613996045205241
0.3271313125607261
[NDCG] 0.5722983141483184


In [21]:
test = True

In [26]:
# load queries
if test:
    qlist = []
    qids = {}
    with open(query_name, 'r') as f:
        for i, line in enumerate(f):
            if i == 0:
                print(line)
            else:
                fields = line.strip().split(',') # q_id, text
                qlist.append(fields[1])
                qids[fields[1]] = fields[0]

Query_Index,Query



In [27]:
if test:
    with open(outcsv_name, 'w') as writer:
        writer.write("Query_Index")
        for i in range(MAXCAND):
            writer.write(",Rank_{:03d}".format(i+1))
   
        for j,qtext in enumerate(qlist):
            print("Query {}".format(j+1))
            
            scores = match(qtext)

            writer.write('\n'+qids[qtext])
            for doc_id, s in scores:
                writer.write(','+doc_id)

Query 1
Query 2
Query 3
Query 4
Query 5
Query 6
Query 7
Query 8
Query 9
Query 10
Query 11
Query 12
Query 13
Query 14
Query 15
Query 16
Query 17
Query 18
Query 19
Query 20
